<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 15px; height: 80px">


# Web Scraping from Indeed.com and Predicting Salaries

### Create a classification model to predict High/Low salary. 


#### Model taking into account job levels and categories:

- Create a few new variables in your dataframe to represent interesting features of a job title.
- For example, create a feature that represents whether 'Senior' is in the title or whether 'Manager' is in the title. 
- Incorporate other text features from the title or summary that you believe will predict the salary.
- Then build new classification models including also those features. Do they add any value? 
- Tune your models by testing parameter ranges, regularization strengths, etc. Discuss how that affects your models. 
- Discuss model coefficients or feature importances as applicable.


#### Thought experiment: What is the baseline accuracy for this model?

baseline_accuracy = majority_class_N / total_N

In [10]:
y.value_counts(normalize=True)

0    0.502762
1    0.497238
Name: USDSalary, dtype: float64

In [11]:
print('baseline:', y.value_counts(normalize=True).max())

baseline: 0.5027624309392266


## Model taking into account job levels and categories

In [43]:
#Create a few new variables in your dataframe to represent interesting features of a job title.
#For example, create a feature that represents whether 'Senior' is in the title or whether 'Manager' is in the title.

In [44]:
df['Jobtitle'].value_counts().head(12)

Data Scientist                   67
Data Engineer                    18
Senior Data Scientist            15
Lead Data Scientist              10
Senior Software Engineer          7
Principal Data Scientist          5
Data Science Manager              4
Machine Learning Engineer         4
Senior Data Engineer              4
Senior Computational Linguist     3
Big Data Scientist                3
Sr. Data Scientist                3
Name: Jobtitle, dtype: int64

In [45]:
df[df['Jobtitle'].str.contains("Senior")].head(2)

,Jobtitle,Location,Salary,Company,Cities,USDSalary
9,Senior Machine Learning Expert/ Data Scientist,"Philadelphia, PA",110000,Workbridge Associates,Philadelphia,110000.0
12,Senior Data Scientist/Senior Machine Learning ...,"Norristown, PA",120000,Workbridge Associates,Philadelphia,120000.0


In [46]:
print(df[df['Jobtitle'].str.contains("Senior")].shape)
print(df[df['Jobtitle'].str.contains("Junior")].shape)
print(df[df['Jobtitle'].str.contains("Engineer")].shape)
print(df[df['Jobtitle'].str.contains("Machine Learning")].shape)
print(df[df['Jobtitle'].str.contains("Lead")].shape)
print(df[df['Jobtitle'].str.contains("Head")].shape)

(146, 6)
(6, 6)
(182, 6)
(44, 6)
(39, 6)
(9, 6)


In [47]:
#Creating a feature that represents whether 'Senior', 'Head' or whether 'Lead' is in the title.

In [48]:
def to_spec(row):
    if "Senior" in row['Jobtitle']:
        return 'Higher title'
    if "Head" in row['Jobtitle']:
        return 'Higher title'
    elif "Lead" in row['Jobtitle']:
        return 'Higher title'
    else:
        return 'Not higher title'


df['Higher position'] = df.apply(to_spec, axis=1)
df.head()


,Jobtitle,Location,Salary,Company,Cities,USDSalary,Higher position
0,Graduate Environmental Scientist,"Philadelphia, PA",47783,City of Philadelphia,Philadelphia,47783.0,Not higher title
1,Environmental Scientist Specialist,"Philadelphia, PA",56589,City of Philadelphia,Philadelphia,56589.0,Not higher title
2,"Data Scientist/ Python, R","Bala-Cynwyd, PA",110000,Workbridge Associates,Philadelphia,110000.0,Not higher title
3,Data Scientist,"Philadelphia, PA",90000,Jobspring Partners,Philadelphia,90000.0,Not higher title
4,"Data Scientist/Analyst (R, Python,Google Cloud)","Philadelphia, PA",90000,Workbridge Associates,Philadelphia,90000.0,Not higher title


In [49]:
#Transforming the classifier to dummy variable.
title_city_position_dum = pd.get_dummies(df[['Cities', 'Higher position', 'Jobtitle']])
title_city_position_dum.head(2)

,Cities_Atlanta,Cities_Austin,Cities_Boston,Cities_Chicago,Cities_Dallas,Cities_Denver,Cities_Houston,"Cities_London, Greater London",Cities_Los+Angeles,Cities_Miami,...,Jobtitle_University Graduate Hire - Software & QA Engineer (SS&S),Jobtitle_Visualisation Consultant,Jobtitle_Visualization Engineer,Jobtitle_Water Quality Assessment Scientist (Natural Resource Scienti...,Jobtitle_Water Quality Monitoring/Data Analysis Specialist (Environme...,Jobtitle_Water Resources Modeler,Jobtitle_Water quality scientist,Jobtitle_Zillow Offers - Software Development Engineer,Jobtitle_iOS Engineer,"Jobtitle_principal data scientist, data & measurement science, Custom..."
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
Xt = title_city_position_dum

In [51]:
logreg = LogisticRegression()
scores = cross_val_score(logreg, Xt, y, cv=10)
print(scores)
print(np.mean(scores))

[0.54945055 0.10989011 0.51648352 0.38461538 0.15384615 0.64444444
 0.64444444 0.46666667 0.5        0.5       ]
0.446984126984127


In [52]:
Xt_train, Xt_test, y_train, y_test = train_test_split(Xt, y, test_size=0.3, random_state=1)
logreg.fit(Xt_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [53]:
from sklearn.preprocessing import StandardScaler
#scaler = StandardScaler()

#Xt_train = scaler.fit_transform(Xt_train)
#Xt_test = scaler.transform(Xt_test)

In [54]:
print(logreg.fit(Xt_train, y_train))
print(logreg.predict(Xt_test))
#print(logreg.predict_proba(Xm_test))

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
[1 0 1 0 1 1 1 1 1 0 1 0 1 0 0 0 1 0 1 1 1 0 0 0 1 0 1 1 1 1 0 1 1 0 1 1 1
 0 1 1 1 0 0 0 1 0 0 0 1 0 1 1 1 1 1 1 1 1 1 0 1 1 0 1 0 1 0 0 1 0 1 0 0 0
 0 0 1 1 1 0 0 0 1 0 1 0 0 1 1 1 1 1 0 1 1 0 1 1 0 1 0 0 1 0 1 0 1 1 1 1 1
 0 0 1 0 1 1 0 0 0 1 0 1 0 1 1 1 0 0 1 0 0 0 0 1 0 1 1 0 0 0 1 1 1 1 0 1 0
 1 1 0 1 0 1 0 1 1 1 1 0 0 1 1 1 0 1 0 1 0 0 1 0 0 1 0 0 0 1 1 0 1 1 1 1 0
 0 0 1 1 0 0 0 1 1 1 0 0 1 0 1 1 1 1 1 0 0 0 1 1 1 1 0 1 1 1 0 1 0 0 0 1 0
 0 1 1 1 0 1 1 1 0 0 1 1 0 0 1 1 1 1 0 0 0 1 1 1 1 1 1 0 0 0 0 0 1 1 0 0 0
 1 1 1 1 0 0 1 1 1 1 1 1 1]


In [55]:
# model score 
logreg.score(Xt,y)

0.8

In [56]:
logreg.score(Xt_train,y_train)

0.8309636650868878

In [57]:
# model score after cross_validation
logreg.score(Xt_test,y_test)

0.7279411764705882

In [58]:
from sklearn.linear_model import LogisticRegressionCV
lr_ridge = LogisticRegressionCV(
    penalty='l2', Cs=100, cv=10, scoring='accuracy')
lr_ridge.fit(Xt_train, y_train)

print(lr_ridge.C_)
print(lr_ridge.score(Xt_train, y_train))
print(lr_ridge.score(Xt_test, y_test))

[2.7825594]
0.9636650868878357
0.7352941176470589


In [59]:
# setup the grid search
params = {'penalty':['l1','l2'],
          'Cs': [np.logspace(-4,4,10)],
          'fit_intercept':[True,False],
          'solver':['liblinear']}

gs = GridSearchCV(estimator=model,
                  param_grid=params,
                  cv=5,
                  scoring='accuracy',
                  return_train_score=True)

gs.fit(Xt_train,y_train)

# extract the grid search results

print('Best Parameters:')
print(gs.best_params_)
print('Best estimator C:')
print(gs.best_estimator_.C_)
print('Best estimator mean cross validated training score:')
print(gs.best_score_)
print('Best estimator score on the full training set:')
print(gs.score(Xt_train,y_train))
print('Best estimator score on the test set:')
print(gs.score(Xt_test,y_test))
#print('Best estimator coefficients:')
#print(gs.best_estimator_.coef_)

Best Parameters:
{'Cs': array([1.00000000e-04, 7.74263683e-04, 5.99484250e-03, 4.64158883e-02,
       3.59381366e-01, 2.78255940e+00, 2.15443469e+01, 1.66810054e+02,
       1.29154967e+03, 1.00000000e+04]), 'fit_intercept': True, 'penalty': 'l1', 'solver': 'liblinear'}
Best estimator C:
[0.35938137]
Best estimator mean cross validated training score:
0.665086887835703
Best estimator score on the full training set:
0.6761453396524486
Best estimator score on the test set:
0.7095588235294118


In [60]:
#Incorporate other text features from the title or summary that you believe will predict the salary.

In [61]:
df['Company'].value_counts().head(12)

Harnham                                         79
New York City DEPT OF HEALTH/MENTAL HYGIENE     29
Jobspring Partners                              23
Workbridge Associates                           23
All-In Analytics                                16
MD Anderson Cancer Center                       16
Liberty Mutual                                  12
Harnham US                                      11
New York City DEPT OF ENVIRONMENT PROTECTION     8
The University of Pittsburgh                     8
State of Washington Dept. of Ecology             7
University College London                        7
Name: Company, dtype: int64

In [62]:
#Creating a feature that represents whether 'Harnham' is in the title.

In [63]:
def to_spec_comp(row):
    if "Harnham" in row['Company']:
        return 'Harnham'
    else:
        return 'Not Harnham'


df['Weather Harnham or not'] = df.apply(to_spec_comp, axis=1)
df.head(2)

,Jobtitle,Location,Salary,Company,Cities,USDSalary,Higher position,Weather Harnham or not
0,Graduate Environmental Scientist,"Philadelphia, PA",47783,City of Philadelphia,Philadelphia,47783.0,Not higher title,Not Harnham
1,Environmental Scientist Specialist,"Philadelphia, PA",56589,City of Philadelphia,Philadelphia,56589.0,Not higher title,Not Harnham


In [64]:
#Transforming the classifier to dummy variable.
title_city_position_dum = pd.get_dummies(df[['Cities', 'Higher position', 'Jobtitle', 'Weather Harnham or not']])
title_city_position_dum.head(2)

,Cities_Atlanta,Cities_Austin,Cities_Boston,Cities_Chicago,Cities_Dallas,Cities_Denver,Cities_Houston,"Cities_London, Greater London",Cities_Los+Angeles,Cities_Miami,...,Jobtitle_Visualization Engineer,Jobtitle_Water Quality Assessment Scientist (Natural Resource Scienti...,Jobtitle_Water Quality Monitoring/Data Analysis Specialist (Environme...,Jobtitle_Water Resources Modeler,Jobtitle_Water quality scientist,Jobtitle_Zillow Offers - Software Development Engineer,Jobtitle_iOS Engineer,"Jobtitle_principal data scientist, data & measurement science, Custom...",Weather Harnham or not_Harnham,Weather Harnham or not_Not Harnham
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [65]:
Xh = title_city_position_dum

In [66]:
logreg = LogisticRegression()
scores = cross_val_score(logreg, Xh, y, cv=10)
print(scores)
print(np.mean(scores))

[0.54945055 0.16483516 0.51648352 0.3956044  0.15384615 0.64444444
 0.53333333 0.4        0.5        0.56666667]
0.4424664224664225


In [67]:
Xh_train, Xh_test, y_train, y_test = train_test_split(Xh, y, test_size=0.3, random_state=1)
logreg.fit(Xh_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [68]:
print(logreg.fit(Xh_train, y_train))
print(logreg.predict(Xh_test))
#print(logreg.predict_proba(Xm_test))

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
[1 0 0 0 1 1 1 1 1 0 0 0 1 0 0 0 0 0 1 1 1 0 0 0 1 0 0 1 1 0 0 0 1 0 1 0 1
 0 1 1 1 0 0 0 0 0 0 0 1 0 1 1 1 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 1 0 1 0 0 0
 0 0 1 1 1 0 0 0 1 0 1 0 0 1 1 1 1 0 0 0 1 0 1 1 0 1 0 0 1 0 1 0 1 1 1 0 1
 0 0 0 0 0 1 0 0 0 1 1 0 0 0 1 1 0 0 1 0 0 0 0 1 0 1 1 0 0 0 1 1 0 0 0 1 0
 1 1 0 1 1 1 0 1 1 1 0 0 0 1 1 1 0 1 0 1 0 0 1 0 0 1 0 0 0 1 1 0 0 1 0 1 0
 0 0 1 1 0 0 0 1 1 1 0 1 0 0 1 1 1 0 1 0 0 0 1 1 1 1 0 1 0 1 0 0 0 0 0 0 0
 0 1 1 1 0 0 1 1 0 0 0 1 0 0 1 1 1 1 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 1 0 0 0
 1 1 1 0 0 0 1 1 1 1 1 1 1]


In [69]:
# model score 
logreg.score(Xh,y)

0.8176795580110497

In [70]:
logreg.score(Xh_train,y_train)

0.8530805687203792

In [71]:
# model score after cross_validation
logreg.score(Xh_test,y_test)

0.7352941176470589

In [72]:
# setup the grid search
params = {'penalty':['l1','l2'],
          'Cs': [np.logspace(-4,4,10)],
          'fit_intercept':[True,False],
          'solver':['liblinear']}

gs = GridSearchCV(estimator=model,
                  param_grid=params,
                  cv=5,
                  scoring='accuracy',
                  return_train_score=True)

gs.fit(Xh_train,y_train)

# extract the grid search results

print('Best Parameters:')
print(gs.best_params_)
print('Best estimator C:')
print(gs.best_estimator_.C_)
print('Best estimator mean cross validated training score:')
print(gs.best_score_)
print('Best estimator score on the full training set:')
print(gs.score(Xh_train,y_train))
print('Best estimator score on the test set:')
print(gs.score(Xh_test,y_test))
#print('Best estimator coefficients:')
#print(gs.best_estimator_.coef_)

Best Parameters:
{'Cs': array([1.00000000e-04, 7.74263683e-04, 5.99484250e-03, 4.64158883e-02,
       3.59381366e-01, 2.78255940e+00, 2.15443469e+01, 1.66810054e+02,
       1.29154967e+03, 1.00000000e+04]), 'fit_intercept': False, 'penalty': 'l1', 'solver': 'liblinear'}
Best estimator C:
[0.35938137]
Best estimator mean cross validated training score:
0.6714060031595577
Best estimator score on the full training set:
0.6966824644549763
Best estimator score on the test set:
0.7058823529411765


In [73]:
#Then build new classification models including also those features. Do they add any value?

In [74]:
#Yes, the score got better everytime I added a new feature, but with different volume, for exapmple 
#when I built a model on only the "cities" and "jobtitles", the score got a lot better as opposed to 
#predicting only on the "cities" feature, so when I added the "high_position" feature the score 
#fall back a little but it was still higher then with only one feature. The highest score reached with 
#the following features: "cities", "jobtitle", "high_position", "spec_company".